In [2]:
import tensorflow as tf

In [3]:
# inputs = tf.keras.Input(shape=(784,), name="digits")
# x = tf.keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
# x = tf.keras.layers.Dense(64, activation="relu", name="dense_2")(x)
# outputs = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)
#
# model = tf.keras.Model(inputs=inputs, outputs=outputs)
# model.summary()

In [4]:
model2 = tf.keras.Sequential(
    [
        tf.keras.Input(shape = (784,), name="digits"),
        tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
        tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
        tf.keras.layers.Dense(10, activation="softmax", name='predictions')
    ]
)
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [6]:
model2.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [7]:
history = model2.fit(
    x_train, y_train,
    batch_size=64, epochs=2,
    validation_data=(x_val, y_val)
)

Epoch 1/2
782/782 [==============================] - 6s 6ms/step - loss: 0.3450 - sparse_categorical_accuracy: 0.9024 - val_loss: 0.1777 - val_sparse_categorical_accuracy: 0.9480

In [8]:
history.history

results = model2.evaluate(x_test, y_test, batch_size=128)
print(results)

79/79 [==============================] - 0s 3ms/step - loss: 0.1322 - sparse_categorical_accuracy: 0.9592
[0.13220123946666718, 0.9592000246047974]


In [9]:
def get_uncompiled_model():
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape = (784,), name="digits"),
            tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
            tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
            tf.keras.layers.Dense(10, activation="softmax", name='predictions')
        ]
    )
    return model

def get_compiled_model():
    model = get_uncompiled_model()

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(),
        loss= tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

In [10]:
# custom losses

# method 01. create custom function with inputs (y_true, y_pred)

def custom_MSE(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true-y_pred))

model3 = get_uncompiled_model()
model3.compile(optimizer=tf.keras.optimizers.Adam(), loss=custom_MSE)

y_train_one_hot = tf.one_hot(y_train, depth=10)
model3.fit(x_train, y_train_one_hot, batch_size =64, epochs=1)


782/782 [==============================] - 3s 4ms/step - loss: 0.0160

In [11]:
# method 02. subclass tf.keras.losses.Loss and create class

class CustomMSE(tf.keras.losses.Loss):
    def __init__(self, regularization_factor = 0.1, name='custom_mse'):
        super().__init__(name = name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor

model4 = get_uncompiled_model()
model4.compile(optimizer= tf.keras.optimizers.Adam(), loss = CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model4.fit(x_train, y_train_one_hot, batch_size =64, epochs =1)

782/782 [==============================] - 4s 4ms/step - loss: 0.0387

In [15]:
# custom metrics
# can create custom metrics by subclassing tf.keras.metrics.Metric

class CategoricalTruePositives(tf.keras.metrics.Metric):
    def __init__(self, name='categorical_true_positives', **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='ctp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1,1))
        values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
        values = tf.cast(values, 'float32')

        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, 'float32')
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_state(self):
        self.true_positives.assign(0.0)


model5 = get_uncompiled_model()
model5.compile(
    optimizer = tf.keras.optimizers.RMSprop(lr=1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = [CategoricalTruePositives()]
)
model5.fit(x_train, y_train, batch_size = 64, epochs= 3)

C:\Users\dev\AppData\Roaming\Python\Python37\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/3
782/782 [==============================] - 5s 6ms/step - loss: 0.3448 - categorical_true_positives: 45068.0000